In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #2 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@411-297> }
00:00:00 #4 [Verbose] > pwd: /home/runner/work/polyglot/polyglot/apps/spiral/temp/test
00:00:00 #5 [Verbose] > dll_path: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:00 #6 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #7 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:00 #8 [Verbose] testPortOpen / ex: System.Aggre

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.5.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @mastoj
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> dotnet restore test.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Paket version 8.0.0-alpha002+6339fce88f07b4d959d03b78330fdb5a6d936483
  The last full restore is still up to date. Nothing left to do.
  Total time taken: 0 milliseconds
  Paket version 8.0.0-alpha002+6339fce88f07b4d959d03b78330fdb5a6d936483
  Restoring /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.csproj
  Starting restore process.
  Total time taken: 0 milliseconds
  Restored /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.csproj (in 259 ms).
Project and references (1 source files) parsed in 4041ms

Started Fable compilation...
Fable compilation finished in 1299ms

./test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bin

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/fable_modules/fable-library-rust)
   Compiling test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
   --> apps/spiral/temp/test/./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^
    |
    = note: `#[warn(unused_imports)]` on by default

  --> apps/spiral/temp/test/./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&mut self, item: &Item) {
   |        ^^^^^^^^^^^
   |
   = note: `#[warn(dead_code)]` on by default

   --> apps/spiral/temp/test/./main.rs:124:4
    |
124 | fn parse_comment(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^^^^^

   --> apps/spiral/temp/test/./main.rs:130:4
    |
130 | fn parse_string(input: &str) -> IResult<&str, Sp

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/fable_modules/fable-library-rust)
   Compiling test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
    Finished release [optimized] target(s) in 13.62s
     Running unittests main.rs (/home/runner/work/polyglot/polyglot/target/release/deps/test-d8be24613d9f5a57)

running 3 tests
test test_parse_number ... ok
2024-02-15T23:07:17.106234Z  INFO test: input=StringLiteral("s:5U~Rf6#1:o)6")
2024-02-15T23:07:17.106299Z  INFO test: input=StringLiteral("kH{}/y}to7+?VG.M&AT{kWD)=")
2024-02-15T23:07:17.106323Z  INFO test: input=Integer(1680720865519160002)
2024-02-15T23:07:17.106371Z  INFO test: input=StringLiteral("igw:~{,'[uiCX=UJ<%$T:VE.")
2024-02-15T23:07:17.106393Z  INFO test: input=Integer(1909268314707122631)
2024-02-15T23:07:17.106420Z  INFO test: input=Comment("2nS>pz(=zV*'<c=*&GB%%.bp9/,{QV")
2024-02-15T23:07:17.106455Z  INFO test: input=Operator("/")
2024-02-15T23:07:17

In [ ]:
{ . $ScriptDir/../../../../target/release/test$(GetExecutableSuffix) } | Invoke-Block

app=test
